In [1]:
import datetime

ipfile = open("input.txt", "r")

Define start and goal state

In [2]:
input = ipfile.readlines()
input

['Start state\n', '1 2 3\n', '4 5 6\n', 'B 7 8']

In [3]:
#obtain start state from input
r1 = input[1]
start = r1.split(' ')
start[2] = start[2].split('\n')[0]
 
r2 = input[2]
start2 = r2.split(' ')
start = start + start2
start[5] = start[5].split('\n')[0]
 
r3 = input[3]
start3 = r3.split(' ')
start = start + start3
start[8] = start[8].split('\n')[0]
 
for n, i in enumerate(start):
  if i == 'B':
    start[n] = 0
 
for i in range(0,9):
  start[i] = int(start[i])

In [4]:
goal = [1, 2, 3, 4, 5, 6, 7, 8, 0]

In [5]:
op = open("output.txt","w")

In [6]:
print(start)
print(goal)

[1, 2, 3, 4, 5, 6, 0, 7, 8]
[1, 2, 3, 4, 5, 6, 7, 8, 0]


In [7]:
op.write("Start state: \n")
print("Start state: \n")
i=0
while i<9:
    if(i%3 == 0):
      print("\n")
      op.write("\n")
    print(str(start[i]) + '   ' + str(start[i+1]) + '   ' + str(start[i+2]))
    op.write(str(start[i]) + '   ' + str(start[i+1]) + '   ' + str(start[i+2]))
    i=i+3
op.write("\n")

Start state: 



1   2   3


4   5   6


0   7   8


1

In [8]:
print("Goal state: \n")
op.write("Goal state: \n")
i=0
while i<9:
    if(i%3 == 0):
      print("\n")
      op.write("\n")
    print(str(goal[i]) + '   ' + str(goal[i+1]) + '   ' + str(goal[i+2]))
    op.write(str(goal[i]) + '   ' + str(goal[i+1]) + '   ' + str(goal[i+2]))
    i=i+3

Goal state: 



1   2   3


4   5   6


7   8   0


Defining Heuristic functions

In [9]:
def getHeuristicMisplacedTiles(start, goal):
  '''Heuristic value uing number of misplaced tiles'''
  h = 0;
  for i in range(0,9):
    if start[i] != goal[i] and start[i]!=0:
      h=h+1
  return h

In [10]:
def getHeuristicManhattanDistance(start, goal):
  '''Heuristic value uing manhattan distance'''
  return sum(abs(b%3 - g%3) + abs(b//3 - g//3)
        for b, g in ((start.index(i), goal.index(i)) for i in range(1, 9)))

In [11]:
L = [] #L is the list of possible states to be explored
Lseen = [] #Lseen is the list of states which have already been explored
cost = []
level = 0
optimalCost = 0

In [12]:
L.append(start) #initially only start state is in L

In [13]:
level = 0
h = getHeuristicMisplacedTiles(start, goal)
cost.append(h)

In [14]:
statesExplored = 0
statesOnOptimalPath = 0

Logic to move a tile in all 4 directions whichever applicable 

In [15]:
def expandNode(state, heuristicType, statesExplored):
  '''Generate all possible neighbours of a state'''
  zerothIndex = state.index(0)
  newStates = [] #[[1,2,3],[2,3,5]]
  finalStates = []
  newState = []  
  if (zerothIndex + 1) % 3 != 0: #move blank to right
    newState = state[:]
    temp = state[zerothIndex + 1]
    newState[zerothIndex + 1] = 0
    newState[zerothIndex] = temp
    newStates.append(newState)
    newState = []
  
  if (zerothIndex % 3) != 0: #move blank to left
    newState = state[:]
    temp = state[zerothIndex - 1]
    newState[zerothIndex - 1] = 0
    newState[zerothIndex] = temp
    newStates.append(newState)
    newState = []

  if zerothIndex not in [0,1,2]: #move blank up
    newState = state[:]
    temp = state[zerothIndex - 3]
    newState[zerothIndex - 3] = 0
    newState[zerothIndex] = temp
    newStates.append(newState)
    newState = []

  if zerothIndex not in [6,7,8]: #move blank down
    newState = state[:]
    temp = newState[zerothIndex + 3]
    newState[zerothIndex + 3] = 0
    newState[zerothIndex] = temp
    newStates.append(newState)
    newState = []

  for newState in newStates:
    statesExplored = statesExplored + 1
    finalStates.append(newState)
    
  return (statesExplored,finalStates)

In [16]:
def getInversionCount(state):
    '''This is used to check if a given state is solveable or not'''
    inversionCount = 0
    for i in range(0,9):
      for j in range(i+1,9):
        if (state[i]>state[j] and state[i]!=0 and state[j]!= 0):
          inversionCount = inversionCount + 1
    print("inversionCount {}".format(inversionCount))
    return inversionCount

Sort the available child states based on the cost

In [17]:
def sortAndPlace(states,L,heuristicType):
  '''Sort the list of states as per their heuristic values'''
  costAndState = {}
  count = 0

  for state in states:
    if heuristicType == 1:
      costAndState[count] = getHeuristicMisplacedTiles(state, goal)
    else:
      costAndState[count] = getHeuristicManhattanDistance(state, goal)
    count = count + 1

  dict1 = dict(sorted(costAndState.items(), key=lambda item: item[1]))
  newList = []
  for key in list(dict1.keys()):
    newList.append(states[key])

  L = newList + L #newList is the sorted list of states as per heuristic value. It is prepended to L
  return L


Actual Hill climbing logic for misplaced tiles

In [18]:
iteration = 0
inv = getInversionCount(start)
if 1 == 0: #inv % 2 != 0:
  print("This puzzle is not solveable")
  op.write("\nThis puzzle is not solveable")
else:
  print("Hill climbing search using Number of misplaced tiles as heuristic\n")
  op.write("\nHill climbing search using Number of misplaced tiles as heuristic\n")
  stateToExpand = L[0]
  totalCost = 0
  statesInOptimalPath = 0
  t1 = datetime.datetime.now()
  prevState = start
  #Keep on iterating till the first value of L is not goal state OR if a local maxima or shoulder is acheieved 
  while(L[0] != goal and (getHeuristicMisplacedTiles(L[0],goal) < getHeuristicMisplacedTiles(prevState,goal) or iteration == 0)):
    iteration = iteration + 1
    stateToExpand = L[0]
    totalCost = totalCost + getHeuristicMisplacedTiles(stateToExpand,goal)
    statesInOptimalPath = statesInOptimalPath +1
    print("****************Iteration number {0}****************".format(iteration))
    op.write("\n****************Iteration number {0}****************".format(iteration))
    i = 0
    while i<9:
      if(i%3 == 0):
        print("\n")
        op.write("\n")
      print(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      op.write(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      i=i+3
    print('\n')
    op.write('\n')
    (statesExplored,childStates) = expandNode(stateToExpand,1, statesExplored)#expand current state

    for state in childStates:
      if state in Lseen or state in L:
        statesExplored = statesExplored - 1
        childStates.remove(state) #remove states which have already been processed

    L = sortAndPlace(childStates,L,1)
    L.remove(stateToExpand)
    Lseen.append(stateToExpand)
    prevState = stateToExpand
    if(len(L) == 0):
      print("This puzzle is not solveable using Hill climbing") #If no more states to process then it is not solveable
      op.write("\nThis puzzle is not solveable using Hill climbing")
      break

  if L[0] == goal:
    #print out the goal state as well
    stateToExpand = L[0]
    print("****************Iteration number {0}****************".format(iteration + 1))
    op.write("\n****************Iteration number {0}****************".format(iteration + 1))
    i = 0
    while i<9:
      if(i%3 == 0):
        print("\n")
        op.write("\n")
      print(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      op.write(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      i=i+3

    t2 = datetime.datetime.now()
    t3 = t2 - t1
    print("Total number of states explored: {}".format(statesExplored-1))
    print("Total number of states in optimal path: {}".format(statesInOptimalPath))
    print("Optimal path cost: {}".format(statesInOptimalPath)) #subtracting 1 because cost to reach start state is 0
    print("Time taken for execution: {}".format(t3))
    op.write("\nTotal number of states explored: {}".format(statesExplored-1))
    op.write("\nTotal number of states in optimal path: {}".format(statesInOptimalPath))
    op.write("\nOptimal path cost: {}".format(statesInOptimalPath)) #subtracting 1 because cost to reach start state is 0
    op.write("\nTime taken for execution: {}".format(t3))


  else:
    print('Not solveable by hill climbing')
    op.write('\nNot solveable by hill climbing')


inversionCount 0
Hill climbing search using Number of misplaced tiles as heuristic

****************Iteration number 1****************


1   2   3


4   5   6


0   7   8


****************Iteration number 2****************


1   2   3


4   5   6


7   0   8


****************Iteration number 3****************


1   2   3


4   5   6


7   8   0
Total number of states explored: 3
Total number of states in optimal path: 2
Optimal path cost: 2
Time taken for execution: 0:00:00.001789


In [19]:
L = []
Lseen = []
cost = []
level = 0
optimalCost = 0

In [20]:
L.append(start)

In [21]:
level = 0
h = getHeuristicManhattanDistance(start, goal)
cost.append(h)

In [22]:
statesExplored = 0
statesOnOptimalPath = 0

Actaul Hill Climbing logic for Manhattan distance

In [23]:
#This is exactly same as above only the heuristic type is changed
iteration = 0
inv = getInversionCount(start)
if 1 == 0: #inv % 2 != 0:
  print("This puzzle is not solveable")
  op.write("\nThis puzzle is not solveable")
else:
  print("Hill climbing search using Manhattan distance as heuristic\n")
  op.write("\nHill climbing search using Manhattan distance as heuristic\n")
  stateToExpand = L[0]
  totalCost = 0
  statesInOptimalPath = 0
  t1 = datetime.datetime.now()
  prevState = start
  while(L[0] != goal and (getHeuristicManhattanDistance(L[0],goal) < getHeuristicManhattanDistance(prevState,goal) or iteration == 0)):
    iteration = iteration + 1
    stateToExpand = L[0]
    totalCost = totalCost + getHeuristicManhattanDistance(stateToExpand,goal)
    statesInOptimalPath = statesInOptimalPath +1
    print("****************Iteration number {0}****************".format(iteration))
    op.write("\n****************Iteration number {0}****************".format(iteration))
    i = 0
    while i<9:
      if(i%3 == 0):
        print("\n")
        op.write("\n")
      print(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      op.write(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      i=i+3
    print('\n')
    op.write('\n')
    (statesExplored,childStates) = expandNode(stateToExpand,1, statesExplored)#expand least cost node
    for state in childStates:
      if state in Lseen or state in L:
        statesExplored = statesExplored - 1
        childStates.remove(state)

    L = sortAndPlace(childStates,L,2)
    L.remove(stateToExpand)
    Lseen.append(stateToExpand)
    prevState = stateToExpand
    if(len(L) == 0):
      print("This puzzle is not solveable using Hill climbing")
      op.write("\nThis puzzle is not solveable using Hill climbing")
      break

  if L[0] == goal:
    stateToExpand = L[0]
    print("****************Iteration number {0}****************".format(iteration + 1))
    op.write("\n****************Iteration number {0}****************".format(iteration + 1))
    i = 0
    while i<9:
      if(i%3 == 0):
        print("\n")
        op.write("\n")
      print(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      op.write(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      i=i+3

    t2 = datetime.datetime.now()
    t3 = t2 - t1
    print("Total number of states explored: {}".format(statesExplored-1))
    print("Total number of states in optimal path: {}".format(statesInOptimalPath ))
    print("Optimal path cost: {}".format(statesInOptimalPath))
    print("Time taken for execution: {}".format(t3))

    op.write("\nTotal number of states explored: {}".format(statesExplored-1))
    op.write("\nTotal number of states in optimal path: {}".format(statesInOptimalPath ))
    op.write("\nOptimal path cost: {}".format(statesInOptimalPath))
    op.write("\nTime taken for execution: {}".format(t3))


  else:
    print('Not solveable by hill climbing')
    op.write('\nNot solveable by hill climbing')


inversionCount 0
Hill climbing search using Manhattan distance as heuristic

****************Iteration number 1****************


1   2   3


4   5   6


0   7   8


****************Iteration number 2****************


1   2   3


4   5   6


7   0   8


****************Iteration number 3****************


1   2   3


4   5   6


7   8   0
Total number of states explored: 3
Total number of states in optimal path: 2
Optimal path cost: 2
Time taken for execution: 0:00:00.009674


In [24]:
L = []
Lseen = []
cost = []
level = 0
optimalCost = 0
L.append(start)
level = 0
h = getHeuristicManhattanDistance(start, goal)
cost.append(h)
statesExplored = 0
statesOnOptimalPath = 0

H3=H1+H2

In [25]:
#For h3=h1+h2
#This is exactly same as above only the heuristic type is changed
iteration = 0
inv = getInversionCount(start)
if 1 == 0: #inv % 2 != 0:
  print("This puzzle is not solveable")
else:
  print("Hill climbing search using h3=h1+h2 as heuristic\n")
  stateToExpand = L[0]
  totalCost = 0
  statesInOptimalPath = 0
  t1 = datetime.datetime.now()
  prevState = start
  while(L[0] != goal and (getHeuristicMisplacedTiles(L[0],goal)+ getHeuristicManhattanDistance(L[0],goal) < (getHeuristicMisplacedTiles(prevState,goal)+ getHeuristicManhattanDistance(prevState,goal)) or iteration == 0)):
    iteration = iteration + 1
    stateToExpand = L[0]
    totalCost = totalCost + getHeuristicManhattanDistance(stateToExpand,goal) + getHeuristicMisplacedTiles(stateToExpand,goal)
    statesInOptimalPath = statesInOptimalPath +1
    print("****************Iteration number {0}****************".format(iteration))
    i = 0
    while i<9:
      if(i%3 == 0):
        print("\n")
      print(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      i=i+3
    print('\n')
    (statesExplored,childStates) = expandNode(stateToExpand,1, statesExplored)#expand least cost node
    for state in childStates:
      if state in Lseen or state in L:
        statesExplored = statesExplored - 1
        childStates.remove(state)

    L = sortAndPlace(childStates,L,2)
    L.remove(stateToExpand)
    Lseen.append(stateToExpand)
    prevState = stateToExpand
    if(len(L) == 0):
      print("This puzzle is not solveable using Hill climbing")
      break

  if L[0] == goal:
    stateToExpand = L[0]
    print("****************Iteration number {0}****************".format(iteration + 1))
    i = 0
    while i<9:
      if(i%3 == 0):
        print("\n")
      print(str(stateToExpand[i]) + '   ' + str(stateToExpand[i+1]) + '   ' + str(stateToExpand[i+2]))
      i=i+3

    t2 = datetime.datetime.now()
    t3 = t2 - t1
    print("Total number of states explored: {}".format(statesExplored-1))
    print("Total number of states in optimal path: {}".format(statesInOptimalPath ))
    print("Optimal path cost: {}".format(totalCost-1))
    print("Time taken for execution: {}".format(t3))

  else:
    print('Not solveable by hill climbing')

inversionCount 0
Hill climbing search using h3=h1+h2 as heuristic

****************Iteration number 1****************


1   2   3


4   5   6


0   7   8


****************Iteration number 2****************


1   2   3


4   5   6


7   0   8


****************Iteration number 3****************


1   2   3


4   5   6


7   8   0
Total number of states explored: 3
Total number of states in optimal path: 2
Optimal path cost: 5
Time taken for execution: 0:00:00.005331


In [26]:
op.close()